In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")


In [2]:
kpis = pd.read_csv("simulation_data/gateway_viability_sweep_ag1_.csv", index_col=0)

In [4]:
[col for col in kpis.columns if not "param" in col]

['Net Minting Rate',
 'KPI C',
 'Experiment Name',
 'State Set',
 'Params Set',
 'timestep',
 'KPI E',
 'KPI 1',
 'KPI 3',
 'KPI 11',
 'KPI 14',
 'total_relays',
 'processed_relays',
 'kpi_a',
 'floating_supply',
 'circulating_supply',
 'dao_value_capture',
 'POKT_burned_cummulative',
 'POKT_minted_cummulative',
 'POKT_net_mint_cummulative',
 'burn_rate_cummulative',
 'mint_rate_cummulative',
 'net_mint_rate_cummulative']

In [7]:
kpis["KPI 1"] #Servicer NPV

0     -6.329316e+10
1     -5.458601e+10
2      1.447123e+11
3      1.843516e+11
4      7.686125e+11
           ...     
283    6.589974e+11
284    1.971990e+12
285    1.857029e+12
286    5.925113e+12
287    5.936564e+12
Name: KPI 1, Length: 288, dtype: float64

In [8]:
kpis["KPI 3"] #Gateway NPV

0     -8.185215e+10
1     -8.077186e+10
2     -2.222146e+11
3     -2.714901e+11
4     -8.744452e+11
           ...     
283   -3.402813e+13
284   -3.414151e+13
285   -3.409033e+13
286   -3.440236e+13
287   -3.430726e+13
Name: KPI 3, Length: 288, dtype: float64

In [10]:
kpis["circulating_supply"] # KPI 4: POKT Circulating Supply

0      1.518397e+16
1      1.518153e+16
2      1.518092e+16
3      1.518029e+16
4      1.518670e+16
           ...     
283    1.521606e+16
284    1.522189e+16
285    1.522147e+16
286    1.524091e+16
287    1.524109e+16
Name: circulating_supply, Length: 288, dtype: float64

In [13]:
network_viability_cols = ["KPI 1",  #Servicer NPV
                          "KPI 3",  #Gateway NPV
                          "circulating_supply", 
                          "floating_supply", # available supply?
                          # TODO: find inflation column
                          "dao_value_capture"]
network_viability_df = kpis[network_viability_cols]
network_viability_df.columns

Index(['KPI 1', 'KPI 3', 'circulating_supply', 'floating_supply',
       'dao_value_capture'],
      dtype='object')

In [21]:
(network_viability_df["KPI 1"] > 0.3).mean() >= 0.7

True

## Network Viability

In [ ]:
# Network Viability
# Threshold Inequality 1
# Servicer NPV

bar_s1 = 750 # POKT Threshold Value
bar_s2 = 0.9 # Fraction of Monte Carlo Sims

def servicer_npv_gt_s1_geq_s2(df: pd.DataFrame,
                              col_name = "KPI 1",
                              min_pokt_threshold: float = None,
                              frac_monte_carlo_sims: float = None) -> int:
    if min_pokt_threshold is None:
        min_pokt_threshold = bar_s1
    if frac_monte_carlo_sims is None:
        frac_monte_carlo_sims = bar_s2
    
    threshold_met = (df[col_name] > min_pokt_threshold).mean() >= frac_monte_carlo_sims

    return threshold_met

servicer_npv_gt_s1_geq_s2(network_viability_df)

In [26]:
# Network Viability
# Threshold Inequality 2
# Gateway NPV

bar_t1 = 7500 # Minimum Gateway NPV Threshold
bar_t2 = 0.9 #Monte Carlo Simulation Fraction

def gateway_npv_gt_t1_geq_t2(df: pd.DataFrame,
                              col_name = "KPI 3",
                              min_pokt_threshold: float = None,
                              frac_monte_carlo_sims: float = None) -> int:
    if min_pokt_threshold is None:
        min_pokt_threshold = bar_t1
    if frac_monte_carlo_sims is None:
        frac_monte_carlo_sims = bar_t2
    
    threshold_met = (df[col_name] > min_pokt_threshold).mean() >= frac_monte_carlo_sims

    return threshold_met

gateway_npv_gt_t1_geq_t2(kpis)

False

In [29]:
# Network Viability
# Threshold Inequality 5
# Relationship Between Circulating Supply and Available Supply

v_1 = 0.3

def ratio_circ_to_available_supply_gt_v1(df: pd.DataFrame,
                                         circ_supply_col:str = "circulating_supply",
                                         available_supply_col:str = "floating_supply",
                                         threshold_val:float = None
                                         ):
    if threshold_val is None:
        threshold_val = v_1
    threshold_met = (df[circ_supply_col]/df[available_supply_col]).mean() > threshold_val
    return threshold_met



SyntaxError: expected ':' (2489983902.py, line 10)